In [33]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # to remove words like {The, for , of , in , with ,etc}..because they are of no use to us
import nltk
nltk.download('stopwords')

from nltk.stem.porter import PorterStemmer # brings words to- base from{(loved, loving)=> love}
from sklearn.feature_extraction.text import TfidfVectorizer #convert words to vectors loved = [0.0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [34]:
news_df = pd.read_csv('train.csv')

In [35]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [36]:
#1=> fake news 0=> real news

In [37]:
news_df.shape

(20800, 5)

In [38]:
news_df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [39]:
news_df=news_df.fillna(' ')

In [40]:
news_df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [41]:
news_df['content'] = news_df['author']+ " " + news_df['title']

In [42]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [43]:
news_df['content'][255]

'Pam Key Spicer to Reporter: ’We’re Going to Raise Our Hands Like Big Boys and Girls’ - Breitbart'

In [44]:
#stemming

In [45]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-z]',' ',content)
    #this is used to remove all the symbols and other punctuation marks except for symbols
    #[^a-zA-z] => is a pattern that i am using here as a negation(^) so that only text remains same
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    
    #now we need to remove the stopwords{the,of,for,in,on,...etc}
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    #so in this part we are aplying stemming techninque only on words which are not the stopwords 
    #stemming is the process that brings words to- base from{(loved, loving)=> love}
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [46]:
news_df['content'] = news_df['content'].apply(stemming)

In [47]:
news_df['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [55]:
X=news_df['content'].values
y=news_df['label'].values

In [56]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [57]:
print(X)

  (0, 15697)	0.28485063562728646
  (0, 13480)	0.2565896679337957
  (0, 8915)	0.3635963806326075
  (0, 8636)	0.29212514087043684
  (0, 7698)	0.24785219520671603
  (0, 7012)	0.21874169089359144
  (0, 4979)	0.233316966909351
  (0, 3795)	0.2705332480845492
  (0, 3603)	0.3598939188262559
  (0, 2962)	0.2468450128533713
  (0, 2485)	0.3676519686797209
  (0, 268)	0.27010124977708766
  (1, 16812)	0.30071745655510157
  (1, 6823)	0.1904660198296849
  (1, 5510)	0.7143299355715573
  (1, 3571)	0.26373768806048464
  (1, 2816)	0.19094574062359204
  (1, 2224)	0.3827320386859759
  (1, 1895)	0.15521974226349364
  (1, 1498)	0.2939891562094648
  (2, 15622)	0.41544962664721613
  (2, 9625)	0.49351492943649944
  (2, 5975)	0.3474613386728292
  (2, 5396)	0.3866530551182615
  (2, 3106)	0.46097489583229645
  :	:
  (20797, 13128)	0.2482526352197606
  (20797, 12350)	0.27263457663336677
  (20797, 12144)	0.24778257724396507
  (20797, 10311)	0.08038079000566466
  (20797, 9593)	0.174553480255222
  (20797, 9523)	0.295420

In [58]:
train_test_split =  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [60]:
X_train.shape

(16640, 17141)

In [61]:
X_test.shape

(4160, 17141)

In [63]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [64]:
train_y_pred = model.predict(X_train)
print("train accuracy:",accuracy_score(train_y_pred,y_train))

train accuracy: 0.9867788461538461


In [65]:
test_y_pred = model.predict(X_test)
print("test accuracy:",accuracy_score(test_y_pred,y_test))

test accuracy: 0.9766826923076923


In [69]:
#prediction system

input_data =X_test[10]
prediction = model.predict(input_data)
if prediction[0]==1:
    prinyt('Fake news')
else:
    print('Real News')

Real News


In [70]:
news_df['content'][10]

'aaron klein obama organ action partner soro link indivis disrupt trump agenda'